# FAA: Airport Arrival Demand Chart

#### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles
import numpy as np
import urllib.request, json

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("grid")
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

## Read data

#### What are the top airports?

In [4]:
airports_df = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_the_United_States"
)[0]

In [5]:
airports_list = airports_df.IATACode.to_list()

In [6]:
airports_list

['ATL',
 'DFW',
 'DEN',
 'ORD',
 'LAX',
 'CLT',
 'MCO',
 'LAS',
 'PHX',
 'MIA',
 'SEA',
 'IAH',
 'JFK',
 'EWR',
 'FLL',
 'MSP',
 'SFO',
 'DTW',
 'BOS',
 'SLC',
 'PHL',
 'BWI',
 'TPA',
 'SAN',
 'LGA',
 'MDW',
 'BNA',
 'IAD']

#### Read data for those places

In [7]:
faa_data = []

for airport in airports_list:
    with urllib.request.urlopen(
        f"https://www.fly.faa.gov/aadc/api/airports/{airport}"
    ) as url:
        faa_data.append(json.load(url))

#### What's available for each airport in the list?

In [8]:
faa_data[0].keys()

dict_keys(['name', 'totalFlightCount', 'cancelledFlightCount', 'dateTime', 'month', 'day', 'year', 'defaultAarRate', 'control', 'rates', 'fixes', 'timeBuckets'])

In [9]:
faa_data[0]["totalFlightCount"]

'1179'

In [10]:
faa_data[0]["cancelledFlightCount"]

4

In [11]:
faa_data[0]["dateTime"]

'081959'

In [12]:
faa_data[0]["month"]

'10'

In [13]:
faa_data[0]["day"]

'8'

In [14]:
faa_data[0]["year"]

'2022'

In [15]:
faa_data[0]["defaultAarRate"]

'126'

In [16]:
dicts = []

for i in faa_data:
    for t in i["timeBuckets"]:
        for f in t["flights"]:
            dicts.append(pd.DataFrame(f, index=[0]))

In [17]:
df = pd.concat(dicts).drop_duplicates(subset=["acid", "type"])

In [18]:
len(df)

7860

In [19]:
df[df["origin"] == "LAX"]

,acid,type,origin,etd,destination,eta,ete,departureCenter,majorAirline
0,DAL548,B752,LAX,A08/1521,ATL,A08/1909,228,ZLA,DAL
0,DAL1557,A321,LAX,A08/1656,ATL,E08/2049,230,ZLA,DAL
0,DAL321,A321,LAX,A08/1721,ATL,E08/2115,230,ZLA,DAL
0,DAL367,B752,LAX,A08/1826,ATL,E08/2222,228,ZLA,DAL
0,NKS631,A20N,LAX,L08/1956,ATL,E08/2356,235,ZLA,NKS
...,...,...,...,...,...,...,...,...,...
0,NKS1073,A320,LAX,S09/0202,BNA,E09/0532,0,ZLA,NKS
0,UAL1984,B772,LAX,A08/1535,IAD,A08/1951,261,ZLA,UAL
0,UAL1635,B739,LAX,A08/1959,IAD,E09/0020,266,ZLA,UAL
0,UAL783,B39M,LAX,S08/2346,IAD,E09/0404,0,ZLA,UAL
